#About Dataset
link https://www.kaggle.com/datasets/agirlcoding/all-space-missions-from-1957/data

Who does not love Space?
This DataSet was scraped from https://nextspaceflight.com/launches/past/?page=1 and includes all the space missions since the beginning of Space Race (1957)


**last part is too complex cuz couzing alote of errors**

##Imports

In [1]:
import numpy as np
import pandas as pd
import re, calendar, json, warnings, joblib
from pathlib import Path
from typing import Dict, Any, Tuple

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.inspection import permutation_importance

warnings.filterwarnings("ignore")
import joblib

##Load the data set

In [2]:
df = pd.read_csv("Space_Corrected.csv")


In [3]:
df.head()

,Unnamed: 0.1,Unnamed: 0,Company Name,Location,Datum,Detail,Status Rocket,Rocket,Status Mission
0,0,0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA","Fri Aug 07, 2020 05:12 UTC",Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.0,Success
1,1,1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...","Thu Aug 06, 2020 04:01 UTC",Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success
2,2,2,SpaceX,"Pad A, Boca Chica, Texas, USA","Tue Aug 04, 2020 23:57 UTC",Starship Prototype | 150 Meter Hop,StatusActive,NaN,Success
3,3,3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan","Thu Jul 30, 2020 21:25 UTC",Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.0,Success
4,4,4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA","Thu Jul 30, 2020 11:50 UTC",Atlas V 541 | Perseverance,StatusActive,145.0,Success


##Explore the data

In [4]:
df.isnull().sum()

,0
Unnamed: 0.1,0
Unnamed: 0,0
Company Name,0
Location,0
Datum,0
Detail,0
Status Rocket,0
Rocket,3360
Status Mission,0


In [5]:
df.duplicated().sum()

np.int64(0)

In [6]:
df.shape

(4324, 9)

i think rocket type is imp but it'll blow when we do one hot

In [7]:
# Calculate success rate for each detail
detail_success = df.groupby('Detail')['Status Mission'].apply(lambda x: (x == 'Success').sum() / len(x)).sort_values(ascending=False)

# Display the success rate for each detail
print("Success rate by Detail:")
display(detail_success)

Success rate by Detail:


,Status Mission
Detail,
Zenit-3 SLBF | Spektr-R,1.0
Zenit-3 SLBF | Elektro-L n†­2,1.0
ASLV | SROSS C2,1.0
Zenit-3 SLBF | Elektro-L n†­1,1.0
Zenit-3 SLBF | AngoSat-1,1.0
...,...
SLV-3 | Rohini RS-D1,0.0
SARGE | Launch 4,0.0
SARGE | Launch 3,0.0


##Preprocessing

In [8]:
df = df.drop(columns=["Unnamed: 0.1","Unnamed: 0"])

we just want rocket type which is at the last part of the descriptions

In [9]:
df['Detail'].value_counts().sum()

np.int64(4324)

In [10]:
df["Rocket_type"] = df["Detail"].astype(str).apply(
    lambda x: x.split("|")[0].strip() if pd.notna(x) else x
)

In [11]:
df["Rocket_type"].value_counts()

,count
Rocket_type,
Cosmos-3M (11K65M),446
Voskhod,299
Molniya-M /Block ML,128
Cosmos-2I (63SM),126
Soyuz U,125
...,...
Long March 2D/YZ-3,1
Redstone Sparta,1
Atlas-SLV3 Agena-B,1


giving unique id to rocket types cuz its important

In [12]:
from sklearn.preprocessing import LabelEncoder

# Extract rocket type first (as we discussed)
df["Rocket_type"] = df["Detail"].astype(str).apply(
    lambda x: x.split("|")[0].strip() if pd.notna(x) else x
)

# Encode to numbers
le = LabelEncoder()
df["Rocket_type_id"] = le.fit_transform(df["Rocket_type"])

# check mapping
mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print("Rocket type mapping:\n", mapping)
print(df[["Rocket_type", "Rocket_type_id"]].head(10))

Rocket type mapping:
 {'ASLV': np.int64(0), 'Angara 1.2': np.int64(1), 'Angara A5/Briz-M': np.int64(2), 'Antares 110': np.int64(3), 'Antares 120': np.int64(4), 'Antares 130': np.int64(5), 'Antares 230': np.int64(6), 'Antares 230+': np.int64(7), 'Ares 1-X': np.int64(8), 'Ariane 1': np.int64(9), 'Ariane 2': np.int64(10), 'Ariane 3': np.int64(11), 'Ariane 40': np.int64(12), 'Ariane 42L': np.int64(13), 'Ariane 42P': np.int64(14), 'Ariane 44L': np.int64(15), 'Ariane 44LP': np.int64(16), 'Ariane 44P': np.int64(17), 'Ariane 5 ECA': np.int64(18), 'Ariane 5 ES': np.int64(19), 'Ariane 5 G': np.int64(20), 'Ariane 5 G+': np.int64(21), 'Ariane 5 GS': np.int64(22), 'Athena I': np.int64(23), 'Athena II': np.int64(24), 'Atlas I': np.int64(25), 'Atlas II': np.int64(26), 'Atlas IIA': np.int64(27), 'Atlas IIAS': np.int64(28), 'Atlas IIIA': np.int64(29), 'Atlas IIIB': np.int64(30), 'Atlas SLV-3': np.int64(31), 'Atlas V 401': np.int64(32), 'Atlas V 411': np.int64(33), 'Atlas V 421': np.int64(34), 'Atlas V 

In [13]:
df.head()

,Company Name,Location,Datum,Detail,Status Rocket,Rocket,Status Mission,Rocket_type,Rocket_type_id
0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA","Fri Aug 07, 2020 05:12 UTC",Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.0,Success,Falcon 9 Block 5,131
1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...","Thu Aug 06, 2020 04:01 UTC",Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success,Long March 2D,167
2,SpaceX,"Pad A, Boca Chica, Texas, USA","Tue Aug 04, 2020 23:57 UTC",Starship Prototype | 150 Meter Hop,StatusActive,NaN,Success,Starship Prototype,293
3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan","Thu Jul 30, 2020 21:25 UTC",Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.0,Success,Proton-M/Briz-M,233
4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA","Thu Jul 30, 2020 11:50 UTC",Atlas V 541 | Perseverance,StatusActive,145.0,Success,Atlas V 541,39


Location and location site also important i extracted that also to get every pice of info

In [14]:
df["Location_country"] = df["Location"].astype(str).apply(
    lambda x: x.split(",")[-1].strip() if pd.notna(x) else x
)

df["Location_site"] = df["Location"].astype(str).apply(
    lambda x: x.split(",")[-2].strip() if (pd.notna(x) and len(x.split(",")) > 1) else x
)


In [15]:
df =df.drop("Location",axis=1)

In [16]:
df.head()

,Company Name,Datum,Detail,Status Rocket,Rocket,Status Mission,Rocket_type,Rocket_type_id,Location_country,Location_site
0,SpaceX,"Fri Aug 07, 2020 05:12 UTC",Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.0,Success,Falcon 9 Block 5,131,USA,Florida
1,CASC,"Thu Aug 06, 2020 04:01 UTC",Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success,Long March 2D,167,China,Jiuquan Satellite Launch Center
2,SpaceX,"Tue Aug 04, 2020 23:57 UTC",Starship Prototype | 150 Meter Hop,StatusActive,NaN,Success,Starship Prototype,293,USA,Texas
3,Roscosmos,"Thu Jul 30, 2020 21:25 UTC",Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.0,Success,Proton-M/Briz-M,233,Kazakhstan,Baikonur Cosmodrome
4,ULA,"Thu Jul 30, 2020 11:50 UTC",Atlas V 541 | Perseverance,StatusActive,145.0,Success,Atlas V 541,39,USA,Florida


In [17]:
df = df.drop(columns=["Detail", "Rocket_type"])

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4324 entries, 0 to 4323
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Company Name      4324 non-null   object
 1   Datum             4324 non-null   object
 2   Status Rocket     4324 non-null   object
 3    Rocket           964 non-null    object
 4   Status Mission    4324 non-null   object
 5   Rocket_type_id    4324 non-null   int64 
 6   Location_country  4324 non-null   object
 7   Location_site     4324 non-null   object
dtypes: int64(1), object(7)
memory usage: 270.4+ KB


In [19]:
df.head()

,Company Name,Datum,Status Rocket,Rocket,Status Mission,Rocket_type_id,Location_country,Location_site
0,SpaceX,"Fri Aug 07, 2020 05:12 UTC",StatusActive,50.0,Success,131,USA,Florida
1,CASC,"Thu Aug 06, 2020 04:01 UTC",StatusActive,29.75,Success,167,China,Jiuquan Satellite Launch Center
2,SpaceX,"Tue Aug 04, 2020 23:57 UTC",StatusActive,NaN,Success,293,USA,Texas
3,Roscosmos,"Thu Jul 30, 2020 21:25 UTC",StatusActive,65.0,Success,233,Kazakhstan,Baikonur Cosmodrome
4,ULA,"Thu Jul 30, 2020 11:50 UTC",StatusActive,145.0,Success,39,USA,Florida


In [20]:
df.columns = ['Company Name', 'Datum', 'Status Rocket', 'Rocket', 'Status Mission',
       'Rocket_type_id', 'Location_country', 'Location_site']

In [21]:
df['Rocket'] = df['Rocket'].astype(str).apply(lambda x: x.replace(',', '')).astype(np.float32)
df['Rocket'] = df['Rocket'].fillna(df['Rocket'].mean())

In [22]:
df.isnull().sum()

,0
Company Name,0
Datum,0
Status Rocket,0
Rocket,0
Status Mission,0
Rocket_type_id,0
Location_country,0
Location_site,0


In [23]:
df.head()

,Company Name,Datum,Status Rocket,Rocket,Status Mission,Rocket_type_id,Location_country,Location_site
0,SpaceX,"Fri Aug 07, 2020 05:12 UTC",StatusActive,50.000000,Success,131,USA,Florida
1,CASC,"Thu Aug 06, 2020 04:01 UTC",StatusActive,29.750000,Success,167,China,Jiuquan Satellite Launch Center
2,SpaceX,"Tue Aug 04, 2020 23:57 UTC",StatusActive,153.792206,Success,293,USA,Texas
3,Roscosmos,"Thu Jul 30, 2020 21:25 UTC",StatusActive,65.000000,Success,233,Kazakhstan,Baikonur Cosmodrome
4,ULA,"Thu Jul 30, 2020 11:50 UTC",StatusActive,145.000000,Success,39,USA,Florida


handling date and time

In [24]:
def get_year_from_date(date):
    year = re.search(r'[^,]*$', date).group(0)
    year = re.search(r'^\s[^\s]*', year).group(0)
    return np.int16(year)

In [25]:
def get_month_from_date(date):
    month = re.search(r'^[^0-9]*', date).group(0)
    month = re.search(r'\s.*$', month).group(0)
    return month.strip()


In [26]:
df['Year'] = df['Datum'].apply(get_year_from_date)
df['Month'] = df['Datum'].apply(get_month_from_date)
df.drop('Datum', axis=1, inplace=True)

encoding

In [27]:
df["Status Rocket"] = df["Status Rocket"].apply(lambda x : 1 if x == "StatusActive" else 0)

In [28]:
df["Status Rocket"].value_counts()


,count
Status Rocket,
0,3534
1,790


In [29]:
df["Status Mission"].value_counts()

,count
Status Mission,
Success,3879
Failure,339
Partial Failure,102
Prelaunch Failure,4


In [30]:
df.head()

,Company Name,Status Rocket,Rocket,Status Mission,Rocket_type_id,Location_country,Location_site,Year,Month
0,SpaceX,1,50.000000,Success,131,USA,Florida,2020,Aug
1,CASC,1,29.750000,Success,167,China,Jiuquan Satellite Launch Center,2020,Aug
2,SpaceX,1,153.792206,Success,293,USA,Texas,2020,Aug
3,Roscosmos,1,65.000000,Success,233,Kazakhstan,Baikonur Cosmodrome,2020,Jul
4,ULA,1,145.000000,Success,39,USA,Florida,2020,Jul


In [31]:
month_ordering = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

In [32]:
# Given some data, a column of that data, and an ordering of the values in that column,
# perform ordinal encoding on the column and return the result.

def ordinal_encode(data, column, ordering):
    return data[column].apply(lambda x: ordering.index(x))

In [33]:
df['Month'] = ordinal_encode(df, 'Month', month_ordering)

In [34]:
df.head()

,Company Name,Status Rocket,Rocket,Status Mission,Rocket_type_id,Location_country,Location_site,Year,Month
0,SpaceX,1,50.000000,Success,131,USA,Florida,2020,7
1,CASC,1,29.750000,Success,167,China,Jiuquan Satellite Launch Center,2020,7
2,SpaceX,1,153.792206,Success,293,USA,Texas,2020,7
3,Roscosmos,1,65.000000,Success,233,Kazakhstan,Baikonur Cosmodrome,2020,6
4,ULA,1,145.000000,Success,39,USA,Florida,2020,6


##Encoding

In [35]:
def onehot_encode(df, column):
    dummies = pd.get_dummies(df[column])
    df = pd.concat([df, dummies], axis=1)
    df.drop(column, axis=1, inplace=True)
    return df

In [36]:
df = onehot_encode(df, 'Company Name')

In [37]:
df.head()

,Status Rocket,Rocket,Status Mission,Rocket_type_id,Location_country,Location_site,Year,Month,AEB,AMBA,...,SpaceX,Starsem,ULA,US Air Force,US Navy,UT,VKS RF,Virgin Orbit,Yuzhmash,i-Space
0,1,50.000000,Success,131,USA,Florida,2020,7,False,False,...,True,False,False,False,False,False,False,False,False,False
1,1,29.750000,Success,167,China,Jiuquan Satellite Launch Center,2020,7,False,False,...,False,False,False,False,False,False,False,False,False,False
2,1,153.792206,Success,293,USA,Texas,2020,7,False,False,...,True,False,False,False,False,False,False,False,False,False
3,1,65.000000,Success,233,Kazakhstan,Baikonur Cosmodrome,2020,6,False,False,...,False,False,False,False,False,False,False,False,False,False
4,1,145.000000,Success,39,USA,Florida,2020,6,False,False,...,False,False,True,False,False,False,False,False,False,False


##setup

In [38]:

df = df.copy()
df.columns = [c.strip() for c in df.columns]

#TARGET (3 classes, no leakage)

df["Status Mission"] = df["Status Mission"].replace({"Prelaunch Failure": "Failure"})
status_map = {"Success": 0, "Failure": 1, "Partial Failure": 2}
y = (df["Status Mission"].astype(str).str.strip().str.title()
                 .replace({"Succes":"Success","Succesful":"Success"})
                 .map(status_map)).astype(int)

# DROP all target-like columns from features
target_like = [c for c in df.columns if re.match(r"(?i)^status\s*mission", c)] + \
              [c for c in ["Mission Outcome","Outcome","Result","Success?","Failure?"] if c in df.columns]
X_raw = df.drop(columns=list(set(target_like)), errors="ignore").copy()



##SIMPLE FEATURE ENGINEERING

In [39]:
# Ensure numeric
for col in ["Rocket", "Year", "Month", "Rocket_type_id", "Status Rocket"]:
    if col in X_raw.columns:
        X_raw[col] = pd.to_numeric(X_raw[col], errors="coerce")

# Add Decade and EarlyEra flag (failures more common early on)
if "Year" in X_raw.columns:
    X_raw["Decade"] = (X_raw["Year"] // 10) * 10
    X_raw["EarlyEra"] = (X_raw["Year"] < 1990).astype(int)

# If Month is string, map to 1..12
if "Month" in X_raw.columns and X_raw["Month"].dtype == object:
    abbr = list(calendar.month_abbr)[1:]
    abbr_map = {m:i for i,m in enumerate(abbr, start=1)}
    def month_to_id(s):
        s = str(s).strip().title()
        if s in abbr_map: return abbr_map[s]
        try:
            i = list(calendar.month_name).index(s)
            return i
        except ValueError:
            return np.nan
    X_raw["Month"] = X_raw["Month"].apply(month_to_id)

##TIME-BASED SPLIT (no leakage)

In [40]:
years = pd.to_numeric(df["Year"], errors="coerce").astype("Int64")
valid = years.notna() & y.notna()
X_raw, y, years = X_raw.loc[valid], y.loc[valid], years.loc[valid].astype(int)

min_y, max_y = int(years.min()), int(years.max())
train_end, val_end = 2016, 2018

def masks(yrs, tr_end, v_end):
    return yrs <= tr_end, (yrs > tr_end) & (yrs <= v_end), yrs > v_end

tr_m, va_m, te_m = masks(years, train_end, val_end)
if not (tr_m.any() and va_m.any() and te_m.any()):
    q70, q85 = int(np.percentile(years, 70)), int(np.percentile(years, 85))
    if q85 <= q70:
        q70, q85 = int(np.percentile(years, 60)), int(np.percentile(years, 80))
    tr_m, va_m, te_m = masks(years, q70, q85)

X_tr_raw, y_tr = X_raw.loc[tr_m].copy(), y.loc[tr_m]
X_va_raw, y_va = X_raw.loc[va_m].copy(), y.loc[va_m]
X_te_raw, y_te = X_raw.loc[te_m].copy(), y.loc[te_m]

if X_va_raw.empty:
    X_tr_raw, X_va_raw, y_tr, y_va = train_test_split(X_tr_raw, y_tr, test_size=0.15, random_state=42, stratify=y_tr)

print(f"Year range: {min_y}..{max_y} | Train: {X_tr_raw.shape} | Val: {X_va_raw.shape} | Test: {X_te_raw.shape}")


Year range: 1957..2020 | Train: (3943, 65) | Val: (209, 65) | Test: (172, 65)


##ENCODE string categoricals (TRAIN ONLY): Location_country/site

In [41]:
encoders: Dict[str, LabelEncoder] = {}
cat_cols = [c for c in ["Location_country", "Location_site"] if c in X_tr_raw.columns and X_tr_raw[c].dtype == object]

def fit_le(c):
    le = LabelEncoder()
    le.fit(X_tr_raw[c].astype(str).fillna("__NA__"))
    encoders[c] = le

def transform_unseen(series, le):
    vals = series.astype(str).fillna("__NA__")
    known = set(le.classes_)
    unseen = set(vals.unique()) - known
    if unseen:
        if "__NA__" not in known:
            le.classes_ = np.concatenate([le.classes_, np.array(["__NA__"])])
        vals = vals.replace(list(unseen), "__NA__")
    return le.transform(vals)

for c in cat_cols:
    fit_le(c)
    X_tr_raw[c] = transform_unseen(X_tr_raw[c], encoders[c])
    X_va_raw[c] = transform_unseen(X_va_raw[c], encoders[c])
    X_te_raw[c] = transform_unseen(X_te_raw[c], encoders[c])

# Keep ALL numeric features (includes your company 0/1 columns)
def numeric_only(df_):
    out = df_.copy()
    for c in out.columns:
        if out[c].dtype == object:
            out[c] = pd.to_numeric(out[c], errors="coerce")
    return out.select_dtypes(include=[np.number]).copy()

X_tr = numeric_only(X_tr_raw)
X_va = numeric_only(X_va_raw)
X_te = numeric_only(X_te_raw)

assert set(X_tr.columns) == set(X_va.columns) == set(X_te.columns), "Feature mismatch across splits."


##Training

In [42]:
def eval_and_print(name: str, model, Xv, yv, Xt, yt) -> Dict[str, Any]:
    yv_pred = model.predict(Xv)
    yt_pred = model.predict(Xt)
    val_acc = accuracy_score(yv, yv_pred); val_f1 = f1_score(yv, yv_pred, average="macro")
    te_acc  = accuracy_score(yt, yt_pred); te_f1  = f1_score(yt, yt_pred, average="macro")
    print(f"\n=== {name} ===")
    print(f"Validation: Acc {val_acc:.4f} | Macro-F1 {val_f1:.4f}")
    print(classification_report(yv, yv_pred, target_names=["Success","Failure","Partial Failure"], digits=4))
    print(f"TEST: Acc {te_acc:.4f} | Macro-F1 {te_f1:.4f}")
    print(classification_report(yt, yt_pred, target_names=["Success","Failure","Partial Failure"], digits=4))
    print("Confusion Matrix (TEST):\n", confusion_matrix(yt, yt_pred))
    return {"name": name, "val_acc": val_acc, "val_f1": val_f1, "test_acc": te_acc, "test_f1": te_f1, "model": model}

results = []

##model + results

In [43]:

# A) HGB balanced
hgb_bal = HistGradientBoostingClassifier(
    class_weight="balanced",
    learning_rate=0.07, max_iter=500, early_stopping=True, validation_fraction=0.15,
    n_iter_no_change=20, random_state=42
).fit(X_tr, y_tr)
results.append(eval_and_print("HGB (balanced)", hgb_bal, X_va, y_va, X_te, y_te))

# B) HGB with stronger class weights (tune weights if needed)
# class indices: 0=Success, 1=Failure, 2=Partial Failure
hgb_w = HistGradientBoostingClassifier(
    class_weight={0:1.0, 1:10.0, 2:15.0},
    learning_rate=0.07, max_iter=500, early_stopping=True, validation_fraction=0.15,
    n_iter_no_change=20, random_state=42
).fit(X_tr, y_tr)
results.append(eval_and_print("HGB (custom weights 1:10:15)", hgb_w, X_va, y_va, X_te, y_te))

# C) SMOTE + HGB (if imblearn available)
try:
    from imblearn.over_sampling import SMOTE
    sm = SMOTE(random_state=42, k_neighbors=3)
    X_tr_sm, y_tr_sm = sm.fit_resample(X_tr, y_tr)
    hgb_sm = HistGradientBoostingClassifier(
        learning_rate=0.07, max_iter=500, early_stopping=True, validation_fraction=0.15,
        n_iter_no_change=20, random_state=42
    ).fit(X_tr_sm, y_tr_sm)
    results.append(eval_and_print("HGB + SMOTE", hgb_sm, X_va, y_va, X_te, y_te))
except Exception as e:
    print("\n[SMOTE unavailable or failed]", e)

# D) LightGBM (if installed)
try:
    import lightgbm as lgb
    lgbm = lgb.LGBMClassifier(
        objective="multiclass",
        num_class=3,
        learning_rate=0.07,
        n_estimators=900,
        max_depth=-1,
        subsample=0.9,
        colsample_bytree=0.8,
        reg_lambda=0.01,
        class_weight={0:1.0, 1:8.0, 2:12.0},  # tune as needed
        random_state=42
    )
    lgbm.fit(X_tr, y_tr, eval_set=[(X_va, y_va)], eval_metric="multi_logloss", verbose=False)
    results.append(eval_and_print("LightGBM (weighted)", lgbm, X_va, y_va, X_te, y_te))
except Exception as e:
    print("\n[LightGBM unavailable or failed]", e)

# E) XGBoost
try:
    import xgboost as xgb
    # build per-sample weights for y_tr
    cls, cnt = np.unique(y_tr, return_counts=True)
    total = cnt.sum()
    w = {c: total/(len(cls)*cnt[i]) for i,c in enumerate(cls)}  # inverse freq
    sw = y_tr.map(w).values

    xgbm = xgb.XGBClassifier(
        objective="multi:softprob",
        num_class=3,
        learning_rate=0.07,
        n_estimators=900,
        max_depth=7,
        subsample=0.9,
        colsample_bytree=0.8,
        reg_lambda=1.0,
        random_state=42,
        tree_method="hist"
    )
    xgbm.fit(X_tr, y_tr, sample_weight=sw, eval_set=[(X_va, y_va)], verbose=False)
    results.append(eval_and_print("XGBoost (weighted)", xgbm, X_va, y_va, X_te, y_te))
except Exception as e:
    print("\n[XGBoost unavailable or failed]", e)





=== HGB (balanced) ===
Validation: Acc 0.7799 | Macro-F1 0.3104
                 precision    recall  f1-score   support

        Success     0.9364    0.8223    0.8757       197
        Failure     0.0357    0.1250    0.0556         8
Partial Failure     0.0000    0.0000    0.0000         4

       accuracy                         0.7799       209
      macro avg     0.3240    0.3158    0.3104       209
   weighted avg     0.8840    0.7799    0.8275       209

TEST: Acc 0.8081 | Macro-F1 0.3948
                 precision    recall  f1-score   support

        Success     0.9306    0.8535    0.8904       157
        Failure     0.2381    0.3846    0.2941        13
Partial Failure     0.0000    0.0000    0.0000         2

       accuracy                         0.8081       172
      macro avg     0.3896    0.4127    0.3948       172
   weighted avg     0.8674    0.8081    0.8349       172

Confusion Matrix (TEST):
 [[134  16   7]
 [  8   5   0]
 [  2   0   0]]

=== HGB (custom weights

##PICK BEST & SAVE

In [44]:
assert results, "No models ran. Check environment."
res_df = pd.DataFrame([{k:v for k,v in r.items() if k!='model'} for r in results]).sort_values(
    by=["test_f1","test_acc"], ascending=False
)
best = max(results, key=lambda r: (r["test_f1"], r["test_acc"]))
best_model = best["model"]

artifacts = Path("artifacts"); artifacts.mkdir(exist_ok=True)
joblib.dump(
    {"model": best_model, "feature_cols": list(X_tr.columns), "encoders": encoders,
     "info": {"year_range": (min_y, max_y), "split": "time-based"}},
    artifacts/"best_model.joblib"
)
res_df.to_csv(artifacts/"model_comparison.csv", index=False)
print("\n=== MODEL COMPARISON (sorted) ===")
print(res_df.to_string(index=False))
print(f"\nSaved best model → {artifacts/'best_model.joblib'}")
print(f"Saved comparison table → {artifacts/'model_comparison.csv'}")


=== MODEL COMPARISON (sorted) ===
                        name  val_acc   val_f1  test_acc  test_f1
              HGB (balanced) 0.779904 0.310410  0.808140 0.394828
HGB (custom weights 1:10:15) 0.794258 0.312450  0.848837 0.394265
                 HGB + SMOTE 0.937799 0.322634  0.912791 0.365662
          XGBoost (weighted) 0.889952 0.313924  0.883721 0.345963

Saved best model → artifacts/best_model.joblib
Saved comparison table → artifacts/model_comparison.csv


In [45]:
try:
    perm = permutation_importance(best_model, X_te, y_te, n_repeats=10, random_state=42, n_jobs=-1)
    pi = pd.Series(perm.importances_mean, index=X_te.columns).sort_values(ascending=False)
    print("\nPermutation Feature Importances (Top 20):")
    print(pi.head(20))
    pi.to_csv(artifacts/"best_model_perm_importance.csv")
except Exception as e:
    print("\n[Permutation importance unavailable for this model]", e)


Permutation Feature Importances (Top 20):
Status Rocket       0.006395
Decade              0.000000
EarlyEra            0.000000
Year                0.000000
Location_country   -0.008140
Rocket             -0.012791
Rocket_type_id     -0.013953
Month              -0.024419
Location_site      -0.057558
dtype: float64
